## Data Cleaning and PreProcessing for Data Visualization
### - Airlines Delay and Cancellations

#### Importing the required libraries

##### Pandas and numpy for Data Manipulation
##### os to set the working directory
##### airportsdata to fetch the latitude and longitude of the airport

In [2]:
import pandas as pd
import numpy as np
import os
import airportsdata
airports = airportsdata.load('IATA')

##### Check the current working directory

In [2]:
os.getcwd()

'/Users/satyanarayanakakarla/Downloads'

##### Converting the reason of delay to categorical column

In [53]:
def get_delay_code(x):
    if x.CARRIER_DELAY > 0:
        return 'Carrier'
    elif x.WEATHER_DELAY > 0:
        return 'Weather'
    elif x.NAS_DELAY > 0:
        return 'NAS'
    elif x.SECURITY_DELAY > 0:
        return 'Security'
    elif x.LATE_AIRCRAFT_DELAY > 0:
        return 'Aircraft'
    elif x.ARR_DELAY > 0 and x.ARR_DELAY < 15:
        return '< 15 min'

##### Reading all the data files and taking sample of them since they are huge
##### 1. Columns with null values > 75 are removed from the dataset.
##### 2. Sample is representative of the original data with extra weight given to Cancelled rows since they are very low.

In [ ]:
data_dict = {}
year = 2009
while year < 2019:
    data_df = pd.read_csv(f'archive (6)/{year}.csv')
    null_df = pd.DataFrame(data_df.isnull().sum() / data_df.shape[0] * 100)
    null_df = null_df.reset_index().rename(columns={'index': 'ColName', 0: 'Percent'})
    subset_columns = list(null_df[(null_df.Percent > 0) & (null_df.Percent < 5) ].ColName.unique())
    cols_to_drop = list(null_df[null_df.Percent > 75].ColName.unique())
    data_df['ARR_DELAY_CODE'] = data_df[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
    data_df.drop(columns=cols_to_drop, inplace = True)
    data_df.dropna(subset=subset_columns, inplace=True)
    if data_df.CANCELLED.sum() > 0:
        print(f"Total flights Cancelled: {data_df.CANCELLED.sum()}. Year: {year}")
        sample_data_df = data_df.sample(frac=0.20, weights = 'CANCELLED')
    else:
        sample_data_df = data_df.sample(frac=0.20)
    sample_data_df.to_parquet(f'archive (6)/{year}_cleaned_sample.csv')
    data_dict[year] = sample_data_df
    print(f"Done for {year}")
    year = year + 1

##### Concatenating the final data

In [ ]:
final_df = pd.concat(data_dict.values())
final_df.shape

##### Writing the final data to a csv

In [ ]:
final_df.to_csv('archive (6)/final_data.csv')

In [4]:
final_df = pd.read_csv('archive (6)/final_data.csv')

##### Taking a sample again of the final dataset because it is still huge

In [5]:
final_df_1 = final_df.sample(frac=0.20)
final_df_1.shape

(2417241, 23)

##### Getting the airport latitude and longitude

In [6]:
final_df_1['ORG_LAT'] = final_df_1['ORIGIN'].apply(lambda x: airports[x].get('lat') if airports.get(x) is not None else np.nan)
final_df_1['ORG_LON'] = final_df_1['ORIGIN'].apply(lambda x: airports[x].get('lon') if airports.get(x) is not None else np.nan)

final_df_1['DEST_LAT'] = final_df_1['DEST'].apply(lambda x: airports[x].get('lat') if airports.get(x) is not None else np.nan)
final_df_1['DEST_LON'] = final_df_1['DEST'].apply(lambda x: airports[x].get('lon') if airports.get(x) is not None else np.nan)

##### Writing the final output to a cav

In [7]:
final_df_1.to_csv('archive (6)/final_data_1.csv')

##### Read the data

In [3]:
final_df_1 = pd.read_csv('archive (6)/final_data_1.csv')

In [4]:
final_df_1.head()

,Unnamed: 0.1,Unnamed: 0,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ARR_DELAY_CODE,ORG_LAT,ORG_LON,DEST_LAT,DEST_LON
0,7437521,2319920,2015-05-27,AA,1343,DSM,DFW,720.0,727.0,7.0,...,0.0,124.0,118.0,88.0,624.0,< 15 min,41.533973,-93.663072,32.897233,-97.037695
1,1530145,3886590,2010-08-08,US,258,PHX,PHL,2235.0,2231.0,-4.0,...,0.0,269.0,265.0,244.0,2075.0,NaN,33.434278,-112.011583,39.872084,-75.240663
2,5826758,3501907,2013-07-19,VX,964,SFO,SAN,1945.0,1945.0,0.0,...,0.0,90.0,90.0,77.0,447.0,NaN,37.618806,-122.375417,32.733563,-117.189663
3,7679157,2506142,2015-06-08,AA,1249,LAX,ATL,1000.0,1000.0,0.0,...,0.0,267.0,253.0,224.0,1947.0,NaN,33.942496,-118.408049,33.636700,-84.427864
4,4024102,2396636,2012-05-25,MQ,4347,DTW,ORD,1030.0,1022.0,-8.0,...,0.0,80.0,69.0,45.0,235.0,NaN,42.212444,-83.353389,41.976940,-87.908150


In [33]:
final_df_1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'FL_DATE', 'OP_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'ARR_DELAY_CODE', 'ORG_LAT', 'ORG_LON', 'DEST_LAT', 'DEST_LON'],
      dtype='object')

##### Creating columns for hour and minute of Scheduled Arrival, Actual Arrival, Scheduled Departure, Actual Departure

In [39]:
final_df_1['CRS_DEP_TIME_HOUR'] = final_df_1.CRS_DEP_TIME.apply(lambda x: int(x /100))
final_df_1['CRS_DEP_TIME_MINUTE'] = final_df_1.CRS_DEP_TIME.apply(lambda x: x %100)
final_df_1['DEP_TIME_HOUR'] = final_df_1.DEP_TIME.apply(lambda x: int(x /100))
final_df_1['DEP_TIME_MINUTE'] = final_df_1.DEP_TIME.apply(lambda x: x %100)

final_df_1['CRS_ARR_TIME_HOUR'] = final_df_1.CRS_ARR_TIME.apply(lambda x: int(x /100))
final_df_1['CRS_ARR_TIME_MINUTE'] = final_df_1.CRS_ARR_TIME.apply(lambda x: x %100)
final_df_1['ARR_TIME_HOUR'] = final_df_1.ARR_TIME.apply(lambda x: int(x /100))
final_df_1['ARR_TIME_MINUTE'] = final_df_1.ARR_TIME.apply(lambda x: x %100)

In [40]:
final_df_1.head()

,Unnamed: 0.1,Unnamed: 0,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,DEST_LAT,DEST_LON,CRS_DEP_TIME_HOUR,CRS_DEP_TIME_MINUTE,DEP_TIME_HOUR,DEP_TIME_MINUTE,CRS_ARR_TIME_HOUR,CRS_ARR_TIME_MINUTE,ARR_TIME_HOUR,ARR_TIME_MINUTE
0,7437521,2319920,2015-05-27,AA,1343,DSM,DFW,720.0,727.0,7.0,...,32.897233,-97.037695,7,20.0,7,27.0,9,24.0,9,25.0
1,1530145,3886590,2010-08-08,US,258,PHX,PHL,2235.0,2231.0,-4.0,...,39.872084,-75.240663,22,35.0,22,31.0,6,4.0,5,56.0
2,5826758,3501907,2013-07-19,VX,964,SFO,SAN,1945.0,1945.0,0.0,...,32.733563,-117.189663,19,45.0,19,45.0,21,15.0,21,15.0
3,7679157,2506142,2015-06-08,AA,1249,LAX,ATL,1000.0,1000.0,0.0,...,33.636700,-84.427864,10,0.0,10,0.0,17,27.0,17,13.0
4,4024102,2396636,2012-05-25,MQ,4347,DTW,ORD,1030.0,1022.0,-8.0,...,41.976940,-87.908150,10,30.0,10,22.0,10,50.0,10,31.0


##### Drop the index columns 

In [43]:
final_df_1.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [44]:
final_df_1.to_csv('archive (6)/final_data_1.csv')

In [45]:
final_df_1.shape

(2417241, 34)

In [46]:
final_df_1.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
       'AIR_TIME', 'DISTANCE', 'ARR_DELAY_CODE', 'ORG_LAT', 'ORG_LON',
       'DEST_LAT', 'DEST_LON', 'CRS_DEP_TIME_HOUR', 'CRS_DEP_TIME_MINUTE',
       'DEP_TIME_HOUR', 'DEP_TIME_MINUTE', 'CRS_ARR_TIME_HOUR',
       'CRS_ARR_TIME_MINUTE', 'ARR_TIME_HOUR', 'ARR_TIME_MINUTE'],
      dtype='object')

##### Reading the data again to get the cancellation data

In [54]:
data_dict = {}
year = 2009
while year < 2019:
    data_df = pd.read_csv(f'archive (6)/{year}.csv')
    subset_data = data_df[data_df.CANCELLED == 1]
    null_df = pd.DataFrame(subset_data.isnull().sum() / subset_data.shape[0] * 100)
    null_df = null_df.reset_index().rename(columns={'index': 'ColName', 0: 'Percent'})
    subset_columns = list(null_df[(null_df.Percent > 0) & (null_df.Percent < 5) ].ColName.unique())
    cols_to_drop = list(null_df[null_df.Percent > 75].ColName.unique())
    subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
    subset_data.drop(columns=cols_to_drop, inplace = True)
    subset_data.dropna(subset=subset_columns, inplace=True)
    data_dict[year] = subset_data
    print(f"Done for {year}")
    year = year + 1

/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

Done for 2009


/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

Done for 2010


/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

...

/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

Done for 2016


/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

Done for 2017
Done for 2018


/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['ARR_DELAY_CODE'] = subset_data[['ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].apply(lambda x: get_delay_code(x), axis = 1)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273/2824285735.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.drop(columns=cols_to_drop, inplace = True)
/var/folders/3k/r09dxnws45j1x__8rtq396100000gn/T/ipykernel_57273

In [55]:
cancelled_data = pd.concat(data_dict.values())
cancelled_data.shape

(973172, 13)

##### Write the cancellation data to a csv

In [56]:
cancelled_data.to_csv('archive (6)/cancelled_data.csv')